## Introduction:

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

### Date Set:
The DataSet contains 20,000 rows, each row cointains information in below mentioned columns.

* id: The unique identifier from Hacker News for the post
* title: The title of the post
* url: The URL that the posts links to, if it the post has a URL
* num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* num_comments: The number of comments that were made on the post
* author: The username of the person who submitted the post
* created_at: The date and time at which the post was submitted

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Below are a couple examples:

Ask HN: How to improve my personal website?
Ask HN: Am I the only one outraged by Twitter shutting down share counts?
Ask HN: Aby recent changes to CSS that broke mobile?

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting. Below are a couple of examples:

Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'
Show HN: Something pointless I made
Show HN: Shanhu.io, a programming playground powered by e8vm

Our aim is to compare these two types of posts to determine the following:

* Do Ask HN or Show HN receive more comments on average?
* Do posts created at a certain time receive more comments on average?

## 1. Reading and Parsing CSV file.

Let's start by parsing the data in hacker_news.csv in form of list of list.

In [5]:
from csv import reader

opened_file = open("hacker_news.csv")
read_file = reader(opened_file)

hn = list(read_file)

# Displaying first five rows of dataset
for row in hn[:5]:
    print(row)
    print("\n")


['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']




## 2. Removing the Header row.
In order to analyse the data we need to remove the header row from the dataset.
Let's store the header row in seprate list and at the same time remove it from our dataset.

In [6]:
headers = hn[0]
hn = hn[1:]

# Print first five rows to verify if header row is removed.
for row in hn[:5]:
    print(row)
    print("\n")

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




## 3. Filter the ASK HN and Show HN Data
Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

In [8]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    #Check if title starts with ask hn or show hn
    if (title.lower()).startswith("ask hn"):
        ask_posts.append(row)
    elif (title.lower()).startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
        
print("Total Ask Posts: ",len(ask_posts))
print("Total Show Posts:",len(show_posts))
print("Total other Posts:",len(other_posts))

Total Ask Posts:  1744
Total Show Posts: 1162
Total other Posts: 17194


## 4. Finding which Type of posts has Maximum Comments
Now we have seprated the "ask posts" and the "show posts" into list of list named ask_posts and show_posts. We can easily move on to our first query i.e. to Determine if ask posts or show posts receive more comments on average.

In [10]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments

avg_ask_comments = total_ask_comments/ len(ask_posts)
print(avg_ask_comments)
total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)
if avg_show_comments > avg_ask_comments:
    print("Show posts have maximum average comments")
else:
    print("Ask Posts have maximum average comments")

14.038417431192661
10.31669535283993
Ask Posts have maximum average comments


According to our calculations, ask posts have received more comments on average as compared to show posts.Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.


## 5. Creation Time of Ask Post Which attaracts Maximum Comments.

we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

* Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
* Calculate the average number of comments ask posts receive by hour created.

First let's extract out number of comments and time at which post was created in seprate list of list so that it would be easy for us to work on our query.

In [1]:
result_list = []

for row in ask_posts:
    creation_time = row[6]
    comments = int(row[4])
    result_list.append([creation_time,comments])

NameError: name 'ask_posts' is not defined

Now to get the average number of comments in each hour, we need to find the total number of comments of each post in each hour of the day and how many posts were posted in each hour of the day.

To achive it we have created two dictionaries.
* Counts_by_hour : We have stored the number of posts posted in each hour of the day.
* Comments_by_hour : We have stored the total of number of comments received in number of posts in each hour of the day.

After that using the data in above two dictionries, we will calculate the average number of comments received in each hour per post.

In [ ]:
import datetime as dt
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    time = dt.datetime.strptime(row[0],"%m/%d/%Y %H:%M")
    hour = time.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
        
print(count_by_hour)
print(comments_by_hour)

Now we will calculate the average number of comments per post for posts created during each hour of the day. This result set will be stored in the list of lists having hour at first index and average number of comments at second index.

In [ ]:
avg_by_hour = []

for hour in comments_by_hour:
    avg_by_hour.append([hour,comments_by_hour[hour]/ counts_by_hour[hour]])
    
for row in avg_by_hour:
    print(row)
    print("\n")

Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [ ]:
swap_avg_by_hour = []

# Add the index 0 and 1 after swapping to new list.
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1],row[0]])
    
for row in swap_avg_by_hour:
    print(row)
    print("\n")
   
# Still rows are not in ascending order, we will sort them by ascending order of average.
sorted_swap = sorted(swap_avg_by_hour, reverse = True)

for row in sorted_swap[:5]:
    print_row = str.format("{}:00: {:.2f} average comments per post.",row[1],row[0])
    print(print_row)
    print("\n")
    